In [2]:
from dotenv import load_dotenv
from openai import OpenAI, AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput, gen_trace_id
from typing import Dict
import os
import asyncio
from bs4 import BeautifulSoup
import requests


In [3]:
load_dotenv(override=True)

# Explicit variable assignments
openai_api_key     = os.getenv('OPENAI_API_KEY')
google_api_key     = os.getenv('GOOGLE_API_KEY')
deepseek_api_key   = os.getenv('DEEPSEEK_API_KEY')
anthropic_api_key  = os.getenv('ANTHROPIC_API_KEY')
gsearch_api_key    = os.getenv('GSEARCH_API_KEY')
search_engine_id   = os.getenv('SEARCH_ENGINE_ID')

# List of variable names (as strings)
var_names = [
    "openai_api_key", "google_api_key", "deepseek_api_key", "anthropic_api_key",
    "gsearch_api_key", "search_engine_id",
]


for var in var_names:
    value = locals()[var]
    label = var.title()
    if value:
        print(f"{label} exists and begins {value[:4]}")
    else:
        print(f"{label} not set")
        

Openai_Api_Key exists and begins sk-p
Google_Api_Key exists and begins AIza
Deepseek_Api_Key exists and begins sk-3
Anthropic_Api_Key exists and begins sk-a
Gsearch_Api_Key exists and begins AIza
Search_Engine_Id exists and begins b5a9


In [3]:
def fetch_search_links(query: str) -> list[str]:
    """
    Queries the Google Custom Search API and returns a list of formatted URLs.
   
    Returns:
        list[str]: A list of formatted URLs returned by the search engine.
    
    Raises:
        RuntimeError: If the request fails or JSON response is invalid.
    """
    url = 'https://www.googleapis.com/customsearch/v1'
    max_results = 5
    params = {
        'q': query,
        'key': gsearch_api_key,
        'cx': search_engine_id,
        'num': max_results
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        results = response.json()

        items = results.get('items', [])
        links = [item.get('formattedUrl') for item in items if item.get('formattedUrl')]

        return links

    except requests.exceptions.RequestException as e:
        raise RuntimeError(f"Request failed: {e}")
    except ValueError:
        raise RuntimeError("Error parsing JSON response.")

In [4]:
def fetch_website_contents(url: str) -> str:
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/117.0.0.0 Safari/537.36"
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        title = soup.title.get_text(strip=True) if soup.title else "No title found"

        if soup.body:
            for tag in soup.body(["script", "style", "img", "input", "noscript"]):
                tag.decompose()
            text = soup.body.get_text(separator="\n", strip=True)
            text = remove_short_lines(text)
        else:
            text = ""

        return f"Webpage Title:\n{title}\n\nWebpage Contents:\n{text}\n"

    except requests.RequestException as e:
        return f"[ERROR] Failed to fetch {url}: {e}"
    except Exception as e:
        return f"[ERROR] Unexpected error: {e}"
    
def remove_short_lines(text: str)-> str:
    min_words=3
    lines = text.splitlines()
    filtered = [line for line in lines if len(line.split()) >= min_words]
    return "\n".join(filtered)


In [8]:
@function_tool
def deep_search_tool(query: str) -> str:
    links = fetch_search_links(query)
    content = ""
    content = f"{content} \nSource URLs:\n{"\n".join(links)}"
    for link in links:
        content = content + f"\n\n## New Search Source:\nSource Content:\n" + fetch_website_contents(link)
    
    return content
             


In [9]:

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
CLAUDE_BASE_URL = "https://api.anthropic.com/v1/"

claude_client = AsyncOpenAI(base_url=CLAUDE_BASE_URL, api_key=anthropic_api_key)
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
lama_client = AsyncOpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
# gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
claude_model=OpenAIChatCompletionsModel(model="claude-3-7-sonnet-20250219", openai_client=claude_client)
llama_model = OpenAIChatCompletionsModel(model="llama3.2:latest", openai_client=lama_client)
# Gemini model does not work with OpenAi Agents SDK
# gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)


In [17]:
sys_text = """ You are an assistant agent who can answer questions for users. \
            if the user asks you any question, then do not answer directly but \
            use the provided deep search tools only to get more recent information \
            related to the question. \nYou should use the deep_search_tool tool \
            to get a context text which you can use to answer questions. \nBe professional and use \
            the given tool and strictly answer questions using the context given by the tool.\n\
            ##\n\n You should provide at the end of your response all the source urls, for example: 'https://www.website.com/pages'\
         """

# text = "what are the stock market summary in Germany for May 2025?"
text = "what are the Agentic AI trends in 2025?"

In [11]:
llama_agent = Agent(name="llama_agent", instructions=sys_text, model=llama_model, tools=[deep_search_tool])
claudeAgent = Agent(name="claude_agent", instructions=sys_text, model=claude_model, tools=[deep_search_tool])
deepseek_agent = Agent(name="deepseek_agent: deepseek_model", model=deepseek_model, instructions=sys_text, tools=[deep_search_tool])
# for openai models, you need to define the model name in the model parameter, no need to define the client and model
gpt_agent = Agent(name="chat_agent: gpt-4o-mini", model="gpt-4o-mini", instructions=sys_text, tools=[deep_search_tool])


In [ ]:
with trace("searcher_agent_"):
    result = await Runner.run(gpt_agent, text)
    print(result.final_output)

In [ ]:
with trace("searcher_agents"):
    results = await asyncio.gather(
        Runner.run(deepseek_agent, text),
        Runner.run(claudeAgent, text),
        Runner.run(gpt_agent, text),
        Runner.run(llama_agent, text)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")